# IATO Residual Risk Notebook
Operational notebook for computing DAG-based, correlation-aware residual risk using Beta-Binomial posteriors, Monte Carlo aggregation, and causal reasoning.

In [ ]:
import numpy as np
import networkx as nx
from scipy.stats import betabinom
from itertools import product

# Set random seed for reproducibility
np.random.seed(42)

## 1. Define DAG & Correlation Matrix
Create a directed acyclic graph representing dependencies between nodes, and a correlation matrix for joint risk exposure.

In [ ]:
# Example DAG with 5 nodes
nodes = [0,1,2,3,4]
edges = [(0,2), (1,2), (2,3), (3,4)]
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# Correlation weights for edges
rho = np.zeros((len(nodes), len(nodes)))
for i,j in edges:
    rho[i,j] = 0.3  # example correlation weight

print("Adjacency matrix:\n", nx.adjacency_matrix(G).todense())
print("Correlation matrix:\n", rho)

## 2. Define Beta-Binomial Posterior Models per Node
Each node has failure counts drawn from a Beta-Binomial distribution.

In [ ]:
n_trials = np.array([10,12,15,8,20])   # number of trials per node
alpha = np.array([2,3,2,5,3])          # alpha parameters
beta = np.array([5,4,3,2,6])           # beta parameters
M = 1000                               # Monte Carlo draws

# Generate Monte Carlo samples for each node
f_samples = np.zeros((M, len(nodes)))
for i in nodes:
    f_samples[:,i] = betabinom.rvs(n_trials[i], alpha[i], beta[i], size=M)

print("Sample Beta-Binomial failures (first 5 MC draws):\n", f_samples[:5,:])

## 3. Compute Residual Risk per Monte Carlo Draw
Aggregate risk across correlated nodes using the correlation matrix and MC draws.

In [ ]:
R_res = np.zeros(M)
for m in range(M):
    total = 0.0
    for i,j in product(nodes,nodes):
        if i != j:
            total += rho[i,j]*f_samples[m,i]
    R_res[m] = total

print("Residual risk (first 5 MC draws):", R_res[:5])
print("Min, Max, Mean, Std:", R_res.min(), R_res.max(), R_res.mean(), R_res.std())

## 4. Causal Reasoning (Pearl's do-calculus)
Compute effect of interventions using simple back-door adjustment on DAG.

In [ ]:
# Define a simple intervention: do(x2=0) meaning node 2 cannot fail
intervention_node = 2

R_res_do = np.zeros(M)
for m in range(M):
    f_copy = f_samples[m,:].copy()
    f_copy[intervention_node] = 0  # apply intervention
    total = 0.0
    for i,j in product(nodes,nodes):
        if i != j:
            total += rho[i,j]*f_copy[i]
    R_res_do[m] = total

print("Residual risk under do(x2=0) (first 5 draws):", R_res_do[:5])
print("Min, Max, Mean, Std under intervention:", R_res_do.min(), R_res_do.max(), R_res_do.mean(), R_res_do.std())

## 5. Summary
- `f_samples`: Monte Carlo Beta-Binomial samples per node
- `rho`: DAG edge correlation weights
- `R_res`: residual risk aggregation across correlated nodes
- `R_res_do`: residual risk under causal intervention

This notebook implements **actual arrays, simulations, and risk calculations** corresponding to the IATO architecture, without pseudocode.